# Import of packages

In [59]:
import numpy as np  # Algebra and numerical computations
import scipy  # Acientific and technical computing
from sklearn import preprocessing  # Utilities for machine learning
from sklearn_som.som import SOM  # Self-Organizing Map implementation
from scipy import stats  # Statistical functions
from scipy.io import arff  # For reading ARFF (Attribute-Relation File Format) files
from sklearn.decomposition import PCA  # Principal Component Analysis for dimensionality reduction
from sklearn.naive_bayes import GaussianNB  # Gaussian Naive Bayes classifier
from sklearn.metrics import accuracy_score  # Function to calculate accuracy score
from sklearn import metrics  # Various machine learning metrics
import os  # Miscellaneous operating system interfaces
from matplotlib import pyplot as plt  # Plotting library
os.environ['KERAS_BACKEND'] = 'tensorflow'  # Set Keras backend to TensorFlow
import tensorflow as tf  # Deep learning framework
np.float = float  # Small correction to ensure compatibility with skmultiflow
from keras.datasets import cifar10  # CIFAR-10 dataset
from keras.datasets import mnist  # MNIST dataset
from keras.utils import to_categorical  # For converting labels to categorical format
from random import randrange  # Generate random numbers
from sklearn.utils import shuffle  # Utility to shuffle datasets
from tqdm import tqdm  # Progress bar for loops
import math  # Mathematical functions
from keras.optimizers import SGD, Adam  # Stochastic Gradient Descent and Adam optimizers
from keras.models import Sequential  # Sequential model in Keras
from keras import layers  # Layers for building neural networks
import pandas as pd  # Data manipulation and analysis library
from datetime import datetime as dt  # Date and time manipulation

# SOM projections

In [60]:
"""
This code is based in the code of Riley Smith (skearn-som), with some modifications for managing the distances.
"""

class LargeSpaceSOM():
    """
    Default is a 2-D, rectangular grid self-organizing map class using Numpy.
    """
    def __init__(self, m=5, n=5, dim=3, lr=1, sigma=1, max_iter=300, dist_type="Cosine",
                    random_state=None):
        """
        Parameters
        ----------
        m : int, default=3
            The shape along dimension 0 (vertical) of the SOM.
        n : int, default=3
            The shape along dimesnion 1 (horizontal) of the SOM.
        dim : int, default=3
            The dimensionality (number of features) of the input space.
        lr : float, default=1
            The initial step size for updating the SOM weights.
        sigma : float, optional
            Optional parameter for magnitude of change to each weight. Does not
            update over training (as does learning rate). Higher values mean
            more aggressive updates to weights.
        max_iter : int, optional
            Optional parameter to stop training if you reach this many
            interation.
        random_state : int, optional
            Optional integer seed to the random number generator for weight
            initialization. This will be used to create a new instance of Numpy's
            default random number generator (it will not call np.random.seed()).
            Specify an integer for deterministic results.
        """
        # Initialize descriptive features of SOM
        self.m = m
        self.n = n
        self.dim = dim
        self.shape = (m, n)
        self.initial_lr = lr
        self.lr = lr
        self.sigma = sigma
        self.max_iter = max_iter
        self.dist_type=dist_type

        # Initialize weights
        self.random_state = random_state
        
        rng = np.random.default_rng(random_state)
        self.weights = rng.normal(size=(m * n, dim))
        self._locations = self._get_locations(m, n)

        # Set after fitting
        self._inertia = None
        self._n_iter_ = None
        self._trained = False
        
        print("Creation of the SOM grid")
        print("Max of iterations", self.max_iter)
        print("Distance", self.dist_type)
        

        # DONE
    def _get_locations(self, m, n):
        """
        Return the indices of an m by n array.
        """
        return np.argwhere(np.ones(shape=(m, n))).astype(np.int64)

        #
        # Critical funtion
    def _find_bmu(self, x):
        """
        Find the index of the best matching unit for the input vector x.
        """
        # Stack x to have one row per weight
        x_stack = np.stack([x]*(self.m*self.n), axis=0)
        # Calculate distance between x and each weight
        if self.dist_type=="Cosine":
            # Cosine measure: invariant to rotation, independent to vector length.
            #distance=np.sum(np.diag(np.outer(x_stack,self.weights)))/(scipy.linalg.norm(x_stack,2)*scipy.linalg.norm(self.weights,2))           
            #print("Cosine distance computation")
            #print("x_stack shape, self.weights shape",x_stack.shape, self.weights.shape)
            distance=np.ndarray(shape=(1,x_stack.shape[0]))
            #print("Distance", distance)
            for i in range(x_stack.shape[0]):
                # Scipy didn't work requires normalized vectors.
                #distance[0,i]=metrics.pairwise.cosine_similarity([x_stack[i,:]], [self.weights[i,:]])
                distance[0,i]=np.dot(x_stack[i,:],self.weights[i,:])/((np.linalg.norm(x_stack[i,:]))*(np.linalg.norm(self.weights[i,:])))
                #print("i", i, "Obtained distance:", distance[0,i])
                # Compute distance between raw input and weights of each neurons.
            
            #Cosine measure can give negative values, so you can compare the relative strengh 
            #of 2 cosine similarities by looking at the absolute values,
            #just like how you would compare the absolute values of 2 Pearson correlations.
            output_value=np.argmin(np.abs(distance[0,:]))
                        
        else:
            #  Frobenius norm of (a-b)
            # Compute distance between raw input and weights of each neurons.
            distance = np.linalg.norm(x_stack - self.weights, axis=1)
            #print("distance shape",distance.shape)
            output_value=np.argmin(distance)
                
        # Find index of best matching unit
        #print("Distance:", distance)
        #print("Output value:", output_value)
        
        return output_value

    def step(self, x):
        """
        Do one step of training on the given input vector.
        """
        # Stack x to have one row per neuron weight, it create a table with multiple row but all with same content.
        x_stack = np.stack([x]*(self.m*self.n), axis=0)

        # Get index of best matching unit
        bmu_index = self._find_bmu(x)
        
        # Find location of best matching unit
        bmu_location = self._locations[bmu_index,:]

        # Find square distance from each weight to the BMU
        stacked_bmu = np.stack([bmu_location]*(self.m*self.n), axis=0)
        # distance among locations.
        bmu_distance = np.sum(np.power(self._locations.astype(np.float64) - stacked_bmu.astype(np.float64), 2), axis=1)

        # Compute update neighborhood - Mexican hat
        neighborhood = np.exp((bmu_distance / (self.sigma ** 2)) * -1)
        local_step = self.lr * neighborhood

        # Stack local step to be proper shape for update
        local_multiplier = np.stack([local_step]*(self.dim), axis=1)

        # Multiply by difference between input and weights
        delta = local_multiplier * (x_stack - self.weights)

        # Update weights
        self.weights += delta

    def _compute_point_intertia(self, x):
        """
        Compute the inertia of a single point. Inertia defined as distance
        from point to closest cluster center (BMU)
        """
        # Find BMU
        bmu_index = self._find_bmu(x)
        bmu = self.weights[bmu_index]
        # Compute distance (just euclidean distance) from x to bmu
        
        if self.dist_type=="Cosine":
            distance=np.abs(np.dot(x,bmu)/((np.linalg.norm(x))*(np.linalg.norm(bmu))))
        else:
            distance = np.linalg.norm(x - bmu)
           
        return distance

    def fit(self, X, epochs=1, shuffle=False):
        """
        Take data (a tensor of type float64) as input and fit the SOM to that
        data for the specified number of epochs.
        """
        # Count total number of iterations
        global_iter_counter = 0
        n_samples = X.shape[0]
        total_iterations = np.minimum(epochs * n_samples, self.max_iter)

        for epoch in range(epochs):
            # Break if past max number of iterations
            if global_iter_counter > self.max_iter:
                break
            # for streaming data has non sense to shuffle.
            if shuffle:
                rng = np.random.default_rng(self.random_state)
                indices = rng.permutation(n_samples)
            else:
                indices = np.arange(n_samples)

            # Train
            for idx in indices:
                # Break if past max number of iterations
                if global_iter_counter > self.max_iter:
                    break
                # Do one step of training
                self.step(X[idx])
                # Update learning rate
                global_iter_counter += 1
                self.lr = (1 - (global_iter_counter / total_iterations)) * self.initial_lr

                
        print("After epochs")
        # Compute inertia
        inertia = np.sum(np.array([float(self._compute_point_intertia(x)) for x in X]))
        self._inertia_ = inertia
        print("Inertia:",inertia)

        # Set n_iter_ attribute
        self._n_iter_ = global_iter_counter

        # Set trained flag
        self._trained = True

        return

    def predict(self, X):
        """
        Predict cluster for each element in X.
        """
        # Check to make sure SOM has been fit
        if not self._trained:
            raise NotImplementedError('SOM object has no predict() method until after calling fit().')

        # Make sure X has proper shape
        assert len(X.shape) == 2, f'X should have two dimensions, not {len(X.shape)}'
        assert X.shape[1] == self.dim, f'This SOM has dimesnion {self.dim}. Received input with dimension {X.shape[1]}'

        labels = np.array([self._find_bmu(x) for x in X])
        return labels

    def transform(self, X):
        """
        Transform the data X into cluster distance space.
        """
        # Stack data and cluster centers
        X_stack = np.stack([X]*(self.m*self.n), axis=1)
        cluster_stack = np.stack([self.weights]*X.shape[0], axis=0)

        # Compute difference
        diff = X_stack - cluster_stack

        # Take and return norm
        return np.linalg.norm(diff, axis=2)

    def fit_predict(self, X, **kwargs):
        """
        Convenience method for calling fit(X) followed by predict(X).
        """
        # Fit to data
        self.fit(X, **kwargs)

        # Return predictions
        return self.predict(X)

    def fit_transform(self, X, **kwargs):
        """
        Convenience method for calling fit(X) followed by transform(X). Unlike
        in sklearn, this is not implemented more efficiently (the efficiency is
        the same as calling fit(X) directly followed by transform(X)).
        """
        # Fit to data
        self.fit(X, **kwargs)

        # Return points in cluster distance space
        return self.transform(X)
    
    def distance_Clusters(self, x):
        """
        Distance to clusters using cosine
        """

        distance=np.ndarray(shape=(1,self.weights.shape[0]))
        #print("Distance", distance)
        for i in range(self.weights.shape[0]):
            distance[0,i]=np.dot(x,self.weights[i,:])/((np.linalg.norm(x))*(np.linalg.norm(self.weights[i,:])))
                
        # Return points in cluster distance space
        return distance

    def distance_Euclidean_Clusters(self, x):
        """
        Distance to clusters using cosine
        """
        distance=np.ndarray(shape=(1,self.weights.shape[0]))
        for i in range(self.weights.shape[0]):
            # Return points in cluster distance space
            difference = x - self.weights[i,:]
            # Compute the Euclidean distance
            distance[0,i]=np.linalg.norm(difference)
            
        return distance
    
    
    @property
    def cluster_centers_(self):
        return self.weights.reshape(self.m, self.n, self.dim)

    @property
    def inertia_(self):
        if self._inertia_ is None:
            raise AttributeError('SOM does not have inertia until after calling fit()')
        return self._inertia_

    @property
    def n_iter_(self):
        if self._n_iter_ is None:
            raise AttributeError('SOM does not have n_iter_ attribute until after calling fit()')
        return self._n_iter_

# Creation of study cases

In [72]:
#
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
#
def extract(A, B, L, lenght):
    # Find the elements in A that are in L
    mask = np.isin(B, L)
    # Get the indices of these elements
    indices = np.where(mask)[0]
    # Randomly select 1000 indices
    selected_indices = np.random.choice(indices, size=lenght, replace=False)
    # Get the selected elements
    selected_elements = A[selected_indices]
    label_elements=B[selected_indices]
    return selected_elements, label_elements

#
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

type1=[1,3,5,7]
type2=[0,6,9]
type3=[2,4]
type4=[8]
#
# Case study A: mixture between type 1 and type 2.
data_Mnist_A_X = np.zeros(shape=(20000,28,28)) 
data_Mnist_A_y = np.zeros(shape=(20000))
#
# Case study: mixture between type 1 + 2 and type 2 + 4.
data_Mnist_B_X = np.zeros(shape=(20000,28,28))
data_Mnist_B_y = np.zeros(shape=(20000))
#
# Mixture between type 1 and type 2 and type type+type4.
data_Mnist_C_X = np.zeros(shape=(20000,28,28))
data_Mnist_C_y = np.zeros(shape=(20000))
#
####
# Case A
####
length=1000
i=0
while i<19:
    #
    selX,sely=extract(X, y, type1, length)
    data_Mnist_A_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_A_y[i*length:(i+1)*length] = sely[:]
    #plt.imshow(data_Mnist_A_X[i*length+1,:,:])
    #plt.savefig(data_path+"DataSets/Img_Type_1_"+str(i*length+1)+".png",bbox_inches='tight')
    #plt.close()
    #
    i=i+1
    selX,sely=extract(X, y, type2, length)
    data_Mnist_A_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_A_y[i*length:(i+1)*length]=sely[:]
    #print("type 2,  i ", i, data_Mnist_A_y[i*length])
    #plt.imshow(data_Mnist_A_X[i*length+1,:,:])
    #plt.savefig(data_path+"DataSets/Img_Type_2_"+str(i*length+1)+".png",bbox_inches='tight')
    #plt.close()

np.save(data_path+"data_Mnist_A_X.npy",data_Mnist_A_X)
np.save(data_path+"data_Mnist_A_y.npy",data_Mnist_A_y)
#plt.imshow(data_Mnist_A_X[16999,:,:])

####
# Case B
####
# Mixture between type 1 + 2 and type 2 + 4.
type1a=[1,2,3,5,7]
type2a=[0,4,6,9]
data_Mnist_B_X = np.zeros(shape=(20000,28,28))
data_Mnist_B_y = np.zeros(shape=(20000))
#
length=1000
i=0
while i<19:
    #
    selX,sely=extract(X, y, type1a, length)
    data_Mnist_B_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_B_y[i*length:(i+1)*length]=sely[:]
    #print("type 1, i ", i, data_Mnist_B_y[i*length])
    #
    i=i+1
    selX,sely=extract(X, y, type2a, length)
    data_Mnist_B_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_B_y[i*length:(i+1)*length]=sely[:]
    #print("type 2,  i ", i, data_Mnist_B_y[i*length])
    #

data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
np.save(data_path+"data_Mnist_B_X.npy",data_Mnist_B_X)
np.save(data_path+"data_Mnist_B_y.npy",data_Mnist_B_y)

####
# Case C
####
# Mixture between type 1 + 2 and type 2 + 4.
type4=[8]
type1a=[1,2,3,5,7]
type2a=[0,4,6,9]
# Mixture between type 1 and type 2 and type type+type4.
data_Mnist_C_X = np.zeros(shape=(20000,28,28))
data_Mnist_C_y = np.zeros(shape=(20000))
#
####
# Case C
####
length=500
i=0
while i<38:
    #
    selX,sely=extract(X, y, type1a, length)
    data_Mnist_C_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_C_y[i*length:(i+1)*length]=sely[:]
    #print("type SET A, i ", i, data_Mnist_C_y[i*length])
    #
    i=i+1
    selX,sely=extract(X, y, type4, length)
    data_Mnist_C_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_C_y[i*length:(i+1)*length]=sely[:]
    #print("type 8,  i ", i, data_Mnist_A_y[i*length])
    i=i+1
    selX,sely=extract(X, y, type2a, length)
    data_Mnist_C_X[i*length:(i+1)*length,:,:] = selX[:,:,:]
    data_Mnist_C_y[i*length:(i+1)*length]=sely[:]
    #print("type SET B, i ", i, data_Mnist_A_y[i*length])

    
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
np.save(data_path+"data_Mnist_C_X.npy",data_Mnist_C_X)
np.save(data_path+"data_Mnist_C_y.npy",data_Mnist_C_y)


# TDA analysis

## Projection with SOM

In [77]:
from sklearn import preprocessing
#    
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
#
# Case study
file_X="data_Mnist_C_X.npy"
file_y="data_Mnist_C_y.npy"
case_A_X=np.load(data_path+file_X)
case_A_Y=np.load(data_path+file_y)
case_A_X_flat = case_A_X.reshape((case_A_X.shape[0], -1))
#
# scale to {-1,1} to better performance of SOM
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(case_A_X_flat)
scaled_data[np.where(scaled_data[:,-1]==-1),-1]=0 
print("Min: ", np.min(scaled_data), "max:", np.max(scaled_data))
#
# To define an initial windows for training the som parameters (20\% of total).
training_windows=round(scaled_data.shape[0]*0.2) # (1 first percent of data I use as reference windows for training the SOM)
#
# initialize the som latticce 10 by 10
m=10
n=10
large_som_arch = LargeSpaceSOM(m=10, n=10, dim=scaled_data.shape[1]-1, dist_type="Euclidean", max_iter=50*training_windows)
# Training
large_som_arch.fit(scaled_data[0:training_windows,0:-1])
# Prediction of clusters:
pred_training_data=large_som_arch.predict(scaled_data[0:training_windows,0:-1])
dist_matrix=np.zeros(shape=(scaled_data.shape[0],n*m))
for i in range(scaled_data.shape[0]):
    dist_matrix[i,:]=large_som_arch.distance_Euclidean_Clusters(scaled_data[i,0:-1])
    #dist_matrix[i,:]=large_som_arch.distance_Clusters(scaled_data[i,0:-1])
    
#print("Moment computation")
# Computation of the moments
mom_matrix=np.zeros(shape=(scaled_data.shape[0],4))
for i in range(scaled_data.shape[0]):
    #mom_matrix[i,0]=np.mean(dist_matrix[i,:])
    #mom_matrix[i,1]=scipy.stats.moment(dist_matrix[i,:], moment=[1])  
    #mom_matrix[i,0]=np.std(dist_matrix[i,:])
    #mom_matrix[i,:]=scipy.stats.moment(dist_matrix[i,:], moment=[1,2,3,4])
    aux_moments=scipy.stats.describe(dist_matrix[i,:])
    mom_matrix[i,0]=aux_moments.mean
    mom_matrix[i,1]=aux_moments.variance
    mom_matrix[i,2]=aux_moments.skewness
    mom_matrix[i,3]=aux_moments.kurtosis

    
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
np.save(data_path+"SOM_Case_C_dist_"+ file_X, dist_matrix)
np.save(data_path+"SOM_Case_C_mom_"+ file_X, mom_matrix)

Min:  -1.0 max: 1.0
Creation of the SOM grid
Max of iterations 200000
Distance Euclidean
After epochs
Inertia: 43843.110217648675


In [78]:
## Projection with PCA

In [87]:
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
#
# Case study
caseStudy="C"
file_X="data_Mnist_"+caseStudy+"_X.npy"
file_y="data_Mnist_"+caseStudy+"_y.npy"
case_A_X=np.load(data_path+file_X)
case_A_Y=np.load(data_path+file_y)
case_A_X_flat = case_A_X.reshape((case_A_X.shape[0], -1))
#
# Scaling in a same way than SOM
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(case_A_X_flat)
scaled_data[np.where(scaled_data[:,-1]==-1),-1]=0 
#
# To define an initial windows for training the som parameters (20\% of total).
training_windows=round(scaled_data.shape[0]*0.2) # (1 first percent of data I use as reference windows for training the SOM)
row_som=10
col_som=10
# chunk_size=50
# steps=(scaled_data.shape[0]-training_windows)//chunk_size
# To define an initial windows for training the som parameters (10\% of total).
# To define an initial windows for training the som parameters (20\% of total).
# Initial training
# Compute PCA in a data stream contex
#
# PCA
#
n_components = 4
pca_proj_data = np.zeros(shape=(scaled_data.shape[0]-training_windows, n_components))
pca = PCA(n_components)
pca.fit(scaled_data[0:training_windows,:])
projPCA=pca.transform(scaled_data)
n_clusters=row_som*col_som
kmeans = KMeans(n_clusters, random_state=42)
kmeans.fit(projPCA[0:training_windows,:])
# Get the distance of each point to each cluster center
distances = kmeans.transform(projPCA)
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Projections/'
np.save(data_path+"PCA_Case_"+caseStudy+"_dist_data_Mnist_X.npy",distances)

C:\Users\sebbas\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\sebbas\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=16.
  warnings.warn(


In [81]:
## Projection with KernelPCA

In [91]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/'
# Case study
caseStudy="C"
file_X="data_Mnist_"+caseStudy+"_X.npy"
file_y="data_Mnist_"+caseStudy+"_y.npy"
#
case_A_X=np.load(data_path+file_X)
case_A_Y=np.load(data_path+file_y)
case_A_X_flat = case_A_X.reshape((case_A_X.shape[0], -1))
#
# scale to {-1,1} to better performance of SOM
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
scaled_data = scaler.fit_transform(case_A_X_flat)
scaled_data[np.where(scaled_data[:,-1]==-1),-1]=0 
#
# To define an initial windows for training the som parameters (20\% of total).
training_windows=round(scaled_data.shape[0]*0.2) # (1 first percent of data I use as reference windows for training the SOM)
row_som=10
col_som=10
som_epochs=5
n_components = 4
kpca_proj_data = np.zeros(shape=(scaled_data.shape[0]-training_windows, n_components))
kernel_pca = KernelPCA(n_components, kernel="rbf")
kernel_pca.fit(scaled_data[0:training_windows,:])
projKPCA=kernel_pca.transform(scaled_data)
n_clusters=row_som*col_som
#
kmeans = KMeans(n_clusters, random_state=42)
kmeans.fit(projKPCA[0:training_windows,:])
distances = kmeans.transform(projKPCA)
data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Projections/'
np.save(data_path+"KernelPCA_Case_"+caseStudy+"_dist_data_Mnist_X.npy",distances)

C:\Users\sebbas\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\sebbas\AppData\Local\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=16.
  warnings.warn(


In [92]:
# Reading pvalues - results were made outside notebook bcs Ripser doesn't work.

In [247]:
import ruptures as rpt
plt.figure(figsize=(18,4))
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
#
chunkSizes=[50,100,250]
caseStudies=["A","B","C"]
lengths=[399,199,79]
steps=[20,10,4]
Methods=["SOM","PCA","KernelPCA"]

<Figure size 1800x400 with 0 Axes>

In [248]:
# Read projections:
#
for trial in [0,1,2]:
    for m in [0,1,2]:
        for caseStudy in ["A","B"]:
            data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Projections/'
            pValueSignal=np.zeros(shape=(3,lengths[trial]))
            pValueSignal[0,:]=np.load(data_path+"pValueSOM_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            pValueSignal[1,:]=np.load(data_path+"pValuePCA_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            pValueSignal[2,:]=np.load(data_path+"pValueKernelPCA_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            #
            data_path = "C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Figures/"
            #
            start = 0
            stop = lengths[trial]
            step= steps[trial]
            chunkSize=chunkSizes[trial]
            # Create the numpy array
            injectedChanges = np.arange(start, stop+step, step)-1
            #
            method=Methods[m]
            signal=pValueSignal[m,:]
            #
            plt.figure(figsize=(18,2))
            plt.plot(signal,"-*")
            plt.xlabel("Chunks")
            plt.xlim(0,stop)
            plt.ylabel("p-value")
            titleFig="Case study " + caseStudy + " - "+ method + " projections - Chunk size: " + str(chunkSize)+ " samples"
            plt.title(titleFig)
            #ax.set_yticks([0,5])
            plt.vlines(injectedChanges[1:], ymin=np.min(signal)-0.1, ymax=np.max(signal)+0.1, colors='green', ls='--', lw=1.3, label='vline_multiple',alpha=0.4)
            plt.hlines(0.05,  xmin=0, xmax=stop,colors='red', ls='-', lw=1.3, alpha=0.6)
            fileName=data_path+method+"_Case_"+caseStudy+str(chunkSize)+".png"
            #print(fileName)
            #plt.savefig(fileName,bbox_inches='tight')
            plt.close()
            

# Case study C, drifts are each 500 samples.
chunkSizes=[50,100,250]
caseStudies=["C"]
lengths=[399,199,79]
steps=[10,5,2]
Methods=["SOM","PCA","KernelPCA"]
#
for trial in [0,1,2]:
    for m in [0,1,2]:
        for caseStudy in ["C"]:
            data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Projections/'
            pValueSignal=np.zeros(shape=(3,lengths[trial]))
            pValueSignal[0,:]=np.load(data_path+"pValueSOM_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            pValueSignal[1,:]=np.load(data_path+"pValuePCA_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            pValueSignal[2,:]=np.load(data_path+"pValueKernelPCA_"+str(chunkSizes[trial])+"_Case_"+caseStudy+".npy")
            #
            data_path = "C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Figures/"
            #
            start = 0
            stop = lengths[trial]
            step= steps[trial]
            chunkSize=chunkSizes[trial]
            # Create the numpy array
            injectedChanges = np.arange(start, stop+step, step)-1
            #
            method=Methods[m]
            signal=pValueSignal[m,:]
            #
            plt.figure(figsize=(18,2))
            plt.plot(signal,"-*")
            plt.xlabel("Chunks")
            plt.xlim(0,stop)
            plt.ylabel("p-value")
            titleFig="Case study " + caseStudy + " - "+ method + " projections - Chunk size: " + str(chunkSize)+ " samples"
            plt.title(titleFig)
            #ax.set_yticks([0,5])
            plt.vlines(injectedChanges[1:], ymin=np.min(signal)-0.1, ymax=np.max(signal)+0.1, colors='green', ls='--', lw=1.3, label='vline_multiple',alpha=0.4)
            plt.hlines(0.05,  xmin=0, xmax=stop,colors='red', ls='-', lw=1.3, alpha=0.6)
            fileName=data_path+method+"_Case_"+caseStudy+str(chunkSize)+".png"
            #print(fileName)
            #plt.savefig(fileName,bbox_inches='tight')
            plt.close()


In [162]:
# Count detections


#### data_path = 'C:/Users/sebbas/PycharmProjects/pythonProject/DELTA-Workshop2024/Projections/'
#
# Case study A - Chunk size 50
chunkSizes=[50,100,250]
caseStudies=["A","B","C"]
lengths=[399,199,79]
Methods=["SOM","PCA","KernelPCA"]
length=[79,199,399]
boundA=0.05
boundA=0.1
i=2
j=2
Methods=["SOM","PCA","KernelPCA"]


#pValueSignal[0,:]=np.load(data_path+"pValueSOM_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")
#pValueSignal[1,:]=np.load(data_path+"pValuePCA_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")
#pValueSignal[2,:]=np.load(data_path+"pValueKernelPCA_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")

print(len(np.where(np.load(data_path+"pValueSOM_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")<=0.05)[0]))
print(len(np.where(np.load(data_path+"pValuePCA_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")<=0.05)[0]))
print(len(np.where(np.load(data_path+"pValueKernelPCA_"+str(chunkSizes[i])+"_Case_"+caseStudy+".npy")<=0.05)[0]))
#.

for caseStudy in caseStudies:
    for chunkSize in chunkSizes:
            print(caseStudy, "&", chunkSize, "&",  Methods[0], "&", "- &", len(np.where(np.load(data_path+"pValueSOM_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.05)[0]), "&",len(np.where(np.load(data_path+"pValueSOM_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.1)[0]))
            print(caseStudy, "&",  chunkSize, "&", Methods[1], "&", "- &", len(np.where(np.load(data_path+"pValuePCA_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.05)[0]), "&", len(np.where(np.load(data_path+"pValuePCA_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.1)[0]))
            print(caseStudy, "&",  chunkSize, "&", Methods[2], "&", "- &", len(np.where(np.load(data_path+"pValueKernelPCA_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.05)[0]), "&", len(np.where(np.load(data_path+"pValueKernelPCA_"+str(chunkSize)+"_Case_"+caseStudy+".npy")<=0.1)[0]))